In [1]:
import os
import math
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
import numpy as np
from skimage.restoration import denoise_tv_bregman
from use_gen_map import gen_map
import pickle
# Ensure the required package is installed
#get inputs
spec_file = r"D:\git_MALDI\MALDI\230615PyMT-Fat-CMC-DHB-Pos-1-PyMT-Spectra-TIC.csv"
mass_file = r"D:\git_MALDI\MALDI\MassList-230615PyMT-Fat-CMC-DHB-Pos-1-PyMT.csv"
spot_file = r"D:\git_MALDI\MALDI\230615PyMT-Fat-CMC-DHB-Pos-1-RegionSpots.csv"

#UNCOMMENT THIS IF YOU WANT TO EXTRACT m/z FROM THE SPECTRA FILE
#os.system("grep m/z " + spec_file + " > mz")
    
#print("m/z is written in a file mz.")

f = open(r"D:\git_MALDI\MALDI\my_own\mz", "r")
mz = f.readline()
mz = mz.rstrip("\n")
mz = mz.split(";")
f.close() 

#os.system("rm mz")


found = False
starting_index = 0
ending_index = 0

for i in range(1, len(mz)):
    mz[i] = float(mz[i])
    if (found == False) and (mz[i] >= 700.5):
        starting_index = i
        found = True
    if (found == True) and (mz[i] >= 700.55):
        ending_index = i
        break



In [14]:
print(starting_index, ending_index)

df = pd.read_csv(spec_file, sep=";", comment="#", usecols=[0, starting_index])
print(df.shape)

169604 169618
(36051, 2)


In [3]:
out_dir = r"D:\git_MALDI\MALDI\output_dir2"
outputtype = "png"

#masses = pd.read_csv("masses_ind.csv")

maps_all = dict() 
added_map = np.zeros((1, 1))
v95_max = 0

for index in range(starting_index, ending_index+1):
    #print(j, "-".join(masses.iloc[j, 4].replace("/", ":").split(":")) + masses.iloc[j, 5][1:])
    #map, v95 = gen_map(index, "mz_data.csv", spots_filename)
    added_map, v95 = gen_map(index, mz_data_filename = spec_file, region_spots_filename = spot_file, spectra_sep = ";", spot_sep = ";", space = 40)

    b = added_map.flatten()
    if sum(b > 0) < 1:
        v95_max = 0
    else:
        v95_max = np.percentile(b[b > -1], 95) 

    plt.figure(figsize=(6.4, 5.2))
    ax = sn.heatmap(added_map, vmax = v95_max, square = True, cmap = 'rainbow')
    plt.ylim(0, added_map.shape[0])
    title = "m/z: " +  str(round(mz[index], 4))
    print(index, title)
    ax.set(xlabel = 'x (mm)', ylabel = 'y (mm)', title = title)
    plt.xticks(np.arange(0,201,50), np.arange(0,9,2))
    plt.yticks(np.arange(0,201,50), np.arange(0,9,2))
    plt.xticks(rotation=0)
    #plt.show()
    filename = out_dir + "/" + "mz" + str(round(mz[index], 4)) + "." + outputtype
    plt.savefig(filename)
    print("Saving " + filename)
    # the following three commands clear too many open figures
    plt.cla()
    plt.clf()
    plt.close("all")

     ## denoising
    # record no data region
    no_data = []
    for l2 in range(added_map.shape[0]):
        for n2 in range(added_map.shape[1]):
            if math.isnan(added_map[l2, n2]): 
                no_data.append((l2, n2))
                added_map[l2, n2] = 0
    plt.figure(figsize=(6.4, 5.2))
    after_noise = denoise_tv_bregman(added_map, weight=0.03)

    # restore no data region
    for l2, n2 in no_data:
        after_noise[l2, n2] = np.nan
    b = after_noise.flatten()
    if sum(b > 0) < 1:
        v95_max = 0
    else:
        v95_max = np.percentile(b[b > -1], 95)

    if v95_max > 0:
        maps_all[mz[index]] = after_noise

    ax = sn.heatmap(after_noise, vmax = v95_max, square = True, cmap = 'rainbow')
    plt.ylim(0, added_map.shape[0])
    title = "m/z: " +  str(round(mz[index], 4))
    #print(j, title)
    ax.set(xlabel = 'x (mm)', ylabel = 'y (mm)', title = title)
    plt.xticks(np.arange(0,201,50), np.arange(0,9,2))
    plt.yticks(np.arange(0,201,50), np.arange(0,9,2))
    plt.xticks(rotation=0)
    #plt.show()
    filename = out_dir + "/" + "mz" + str(round(mz[index], 4)) + "_denoised." + outputtype
    plt.savefig(filename)
    print("Saving " + filename)
    # the following three commands clear too many open figures
    plt.cla()
    plt.clf()
    plt.close("all")

    ##
    added_map = np.zeros((1, 1))
    v95_max = 0

with open(r'D:\git_MALDI\MALDI\my_own\saved_dictionary2.pkl', 'wb') as f:
    pickle.dump(maps_all, f)


spectra has been read
output is [map, v95]
169604 m/z: 700.501
Saving D:\git_MALDI\MALDI\output_dir2/mz700.501.png
Saving D:\git_MALDI\MALDI\output_dir2/mz700.501_denoised.png
spectra has been read
output is [map, v95]
169605 m/z: 700.5045
Saving D:\git_MALDI\MALDI\output_dir2/mz700.5045.png
Saving D:\git_MALDI\MALDI\output_dir2/mz700.5045_denoised.png
spectra has been read
output is [map, v95]
169606 m/z: 700.508
Saving D:\git_MALDI\MALDI\output_dir2/mz700.508.png
Saving D:\git_MALDI\MALDI\output_dir2/mz700.508_denoised.png
spectra has been read
output is [map, v95]
169607 m/z: 700.5115
Saving D:\git_MALDI\MALDI\output_dir2/mz700.5115.png
Saving D:\git_MALDI\MALDI\output_dir2/mz700.5115_denoised.png
spectra has been read
output is [map, v95]
169608 m/z: 700.515
Saving D:\git_MALDI\MALDI\output_dir2/mz700.515.png
Saving D:\git_MALDI\MALDI\output_dir2/mz700.515_denoised.png
spectra has been read
output is [map, v95]
169609 m/z: 700.5185
Saving D:\git_MALDI\MALDI\output_dir2/mz700.5185.p